# Assignmnet 2 (100 points)

**Name:** <br>
**Email:** <br>
**Group:** A/B <br>
**Hours spend *(optional)* :** <br>

### SMS Spam Detection *(100 points)*

<p>You are hired as an AI expert in the development department of a telecommunications company. The first thing on your orientation plan is a small project that your boss has assigned you for the following given situation. Your supervisor has given away his private cell phone number on too many websites and is now complaining about daily spam SMS. Therefore, it is your job to write a spam detector in Python. </p>

<p>In doing so, you need to use a Naive Bayes classifier that can handle both bag-of-words (BoW) and tf-idf features as input. For the evaluation of your spam detector, an SMS collection is available as a dataset - this has yet to be suitably split into train and test data. To keep the costs as low as possible and to avoid problems with copyrights, your boss insists on a new development with Python.</p>

<p>Include a short description of the data preprocessing steps, method, experiment design, hyper-parameters, and evaluation metric. Also, document your findings, drawbacks, and potential improvements.</p>

<p>Note: You need to implement the bag-of-words (BoW) and tf-idf feature extractor from scratch. You can use existing python libraries for other tasks.</p>

**Dataset and Resources**

* SMS Spam Collection Dataset: https://archive.ics.uci.edu/dataset/228/sms+spam+collection

In [9]:
import pandas as pd
from collections import Counter
import math
import random
with open ('C:/Users/Administrator/Desktop/Semester3/Sprach-und Textverstehen/Prak2/Assignmnet_2/sms_spam_collection/SMSSpamCollection', 'r') as file1:
    text = file1.read()
    
 
with open ('C:/Users/Administrator/Desktop/Semester3/Sprach-und Textverstehen/Prak2/Assignmnet_2/sms_spam_collection/SMSSpamCollection', 'r') as file2:
    textSentence = file2.readlines()
text = text.split()


"""
Hyperparameter
"""
removingPreandSuffix = ".?!?/!"   
textSentence = textSentence[:]
"""
Creating Training and Test dataset
"""
def split_data(data, test_ratio=0.2):   #Created with Chatgpt!
    data_copy = data[:]  # Kopie der Daten, um das Original nicht zu ändern
    random.shuffle(data_copy)  # Zufälliges Mischen der Daten

    split_index = int(len(data_copy) * (1 - test_ratio))
    train_data = data_copy[:split_index]  # Trainingsteil
    test_data = data_copy[split_index:]   # Testteil

    return train_data, test_data

trainingtextSentence, testtextSentence = split_data(textSentence)

"""
Makes a word list for training and test data
"""
TrainingTextWord = []
for i in trainingtextSentence:
    i = i.split()
    for u in range(0, len(i)):
        TrainingTextWord.append(i[u])
TestTextWord = []
for i in testtextSentence:
    i = i.split()
    for u in range(0, len(i)):
        TestTextWord.append(i[u])

"""
Process the input sentence and 
removes chosen suffix and prefix
"""
def processSentence(trainingtextSentence):
    category = ""
    wordList = []
    sentenceList = []
    for sentence in trainingtextSentence:
        category = sentence[0]
        containNumber = 0
        wort = sentence.split()
        for wort in wort:
            for remover in removingPreandSuffix: 
                wort = wort.casefold()
                wort = wort.removesuffix("...")
                wort = wort.removesuffix("..")
                wort = wort.removesuffix(".")
                wort = wort.removesuffix(remover)
                wort = wort.removeprefix(remover)
            if wort == "" or wort == "ham" or wort == "spam":
                containNumber = 1
            if containNumber == 0:
                wordList.append(wort)        #Add words to the word list
            else:
                    containNumber = 0
        dic = {
            "Class": category,
            "Text": wordList
        }
        sentenceList.append(dic)
        wordList = []
    print("Process Sentence DONE")
    return sentenceList

"""
Process the input word list of unique words and 
removes chosen suffix and prefix
"""
def processSpecialWords(wort):
        wordList = []
        containNumber = 0
        for i in range(0, len(wort)):
                for remover in removingPreandSuffix: 
                        wort[i] = wort[i].casefold()
                        wort[i] = wort[i].removesuffix("...")
                        wort[i] = wort[i].removesuffix("..")
                        wort[i] = wort[i].removesuffix(".")
                        wort[i] = wort[i].removesuffix(remover)
                        wort[i] = wort[i].removeprefix(remover)

                if wort[i] == "" or wort[i] == "ham" or wort[i] == "spam" or wordList.__contains__(wort[i]):
                        containNumber = 1
                if containNumber == 0:
                        wordList.append(wort[i])        #Add words to the word list
                else:
                        containNumber = 0
        print("Process Special Words DONE")
        return wordList

"""
Process the input word list of all words and 
removes chosen suffix and prefix
"""
def processSpecialWordsComplete(wort, frequency= 0):
        wordList = []
        containNumber = 0
        for i in range(0, len(wort)):
                for remover in removingPreandSuffix: 
                        wort[i] = wort[i].casefold()
                        wort[i] = wort[i].removesuffix("...")
                        wort[i] = wort[i].removesuffix("..")
                        wort[i] = wort[i].removesuffix(".")
                        wort[i] = wort[i].removesuffix(remover)
                        wort[i] = wort[i].removeprefix(remover)

                if wort[i] == "" or wort[i] == "ham" or wort[i] == "spam":
                        containNumber = 1
                if containNumber == 0:
                        wordList.append(wort[i])        #Add words to the word list
                else:
                        containNumber = 0
        
        if frequency == 1:
            frequeny = Counter(wordList)
            return frequency
        print("Process Special Words Complete DONE")
        return wordList

"""
Creates the vector for the bag of words. 
Can take idf features to change the weight of the words
"""
def creatingvector(sentenceList, wordList, idf_features=0):
    matrix = [[0] * (len(wordList)+1) for _ in range(len(sentenceList)+1)]
    
    for i in range(len(sentenceList)): # every Sentence
        matrix[i+1][0] = sentenceList[i]
        for s in range(len(wordList)): #every Special Word
            if s == 0:
                matrix[0][s+1] = wordList[s]
            for w in range(len(sentenceList[i]["Text"])): #every Word in Sentence
                if sentenceList[i]["Text"][w] == wordList[s]:
                    if idf_features == 0:
                        matrix[i+1][s+1] = matrix[i+1][s+1] +1
                    else:
            
                        for z in range(0, len(idf_features)):
                            if idf_features[z][1] == wordList[s]:
                                matrix[i+1][s+1] = idf_features[z][0]
                            else:
                                matrix[i+1][s+1] = matrix[i+1][s+1] +1	    
    print("Creatin Vector DONE")                      
    return matrix

"""
Calculates the probability of each word and creates
a list for the Spam and not Spam classification
(very ineffecient)
"""
def probabilityCalculator(matrix, vocabularyWords, SumofWords):
    print("Probability Starts")
    spamCount = 0
    hamCount = 0
    probListSpam = []
    probListHam = []
    allWords = len(vocabularyWords)

    for i in range(0, len(vocabularyWords)):
 
        currentWord = vocabularyWords[i]
        for u in range(0, len(matrix)-1):
           
            if matrix[u+1][0]["Class"] == 'h':
                hamCount = hamCount+matrix[u+1][i+1]
          
            if matrix[u+1][0]["Class"] == 's':
                spamCount = spamCount+matrix[u+1][i+1]
        probH = ((hamCount+1)/(SumofWords[0]+allWords))
        probS = ((spamCount+1)/(SumofWords[1]+allWords))
        dicS = {
            "Class": "Spam",
            "Word": currentWord,
            "Probability": probS
            }
        dicH = {
            "Class": "Ham",
            "Word": currentWord,
            "Probability": probH
            }

        hamCount = 0
        spamCount = 0
        probListSpam.append(dicS)
        probListHam.append(dicH)
    print("Probability DONE")
    return probListHam, probListSpam

"""
Counts the total words in 
the spam and not spam class
"""
def countingWords(matrix):
    sumWordsinClassS = 0
    sumWordsinClassH = 0
    for i in range(0, len(matrix)-1):
        if matrix[i+1][0]["Class"] == "h":
            sumWordsinClassH = sumWordsinClassH +sum(matrix[i+1][1:])
        if matrix[i+1][0]["Class"] == "s":
            sumWordsinClassS = sumWordsinClassS +sum(matrix[i+1][1:])
    return sumWordsinClassH, sumWordsinClassS

"""
Counts the total labels of the spam and not spam
class
"""
def countingLabel(matrix):
    HLabel = 0
    SLabel = 0
    Total = 0
    for i in range(0, len(matrix)-1):
        if matrix[i+1][0]["Class"] == "h":
            HLabel = HLabel+1
        if matrix[i+1][0]["Class"] == "s":
            SLabel = SLabel+1
        Total = Total+1
    return HLabel, SLabel, Total

"""
Predicts the class 
of the sentence
"""
def predictingClass(testMatrix, probListHam, probListSpam, LabelCount, SumofWords, vocabularyWords):
    prioriHam =  LabelCount[0] / LabelCount[2]
    prioriSpam = LabelCount[1] / LabelCount[2]
    probHam = LabelCount[0] / LabelCount[2]
    probSpam = LabelCount[1] / LabelCount[2]
    count = 0
    classifiedHamCounter = 0
    classifiedSpamCounter = 0
    AccuracyHam = 0
    AccuracySpam = 0
    for i in range(0, len(testMatrix)-1):
        count = count+1
        for u in range(0, len(testMatrix[i+1][0]["Text"])):
            for w in range(0, len(probListHam)):
                if testMatrix[i+1][0]["Text"][u] == probListHam[w]["Word"]:
                    probHam = probHam * probListHam[w]["Probability"]
                    break
                elif(w == len(probListHam)-1):
                    probHam = probHam * (1/(SumofWords[0]+len(vocabularyWords)))
            for w in range(0, len(probListSpam)):
                if testMatrix[i+1][0]["Text"][u] == probListSpam[w]["Word"]:
                    probSpam = probSpam * probListSpam[w]["Probability"]
                    break
                elif(w == len(probListSpam)-1):
                    probSpam = probSpam * (1/(SumofWords[0]+len(vocabularyWords)))
        if(probSpam > probHam):
            print("Email"+str(count)+": "+str(testMatrix[count]))
            print("Email"+str(count)+" CLASSIFIED as Spam\n")
            classifiedSpamCounter = classifiedSpamCounter+1
            if testMatrix[i+1][0]["Class"] == "s":
                AccuracySpam = AccuracySpam+1
        if(probHam > probSpam): 
            print("Email"+str(count)+": "+str(testMatrix[count]))
            print("Email"+str(count)+" CLASSIFIED as no Spam\n")
            classifiedHamCounter = classifiedHamCounter+1
            if testMatrix[i+1][0]["Class"] == "h":
                AccuracyHam = AccuracyHam+1
        probHam = prioriHam
        probSpam = prioriSpam

    classCount = countingLabel(testMatrix)
    print(classCount[0])
    print("True Spam Accuracy: ", AccuracySpam/classCount[1]," ,Amount",classCount[1])
    print("True Ham Accuracy: ", AccuracyHam/classCount[0]," ,Amount",classCount[0])
    print("Total Prediction Accuracy: ",(AccuracyHam+AccuracySpam)/classCount[2])

"""
Calculates the termfreqency of words
"""
def termfrequency(wordList):
    frequency = Counter(wordList)
    frequency = sorted(frequency.items(), key=lambda x: x[1])
    wordCount = 0
    frequencyWordList = []
    for i in range(0, len(frequency)):
        wordCount = wordCount+frequency[len(frequency)-1-i][1]
    for i in range(0, len(frequency)):
        frequencyWordList.append([frequency[len(frequency)-1-i][1]/wordCount, frequency[len(frequency)-1-i][0]])
    return frequencyWordList

"""
Calculates the inverse document frequency of
words
"""
def InverseDocumentFrequency(frequencyWordList, sentences):
    amountSentence = len(sentences)
    inDocument = 0
    count = -1
    for i in range(0, len(frequencyWordList)):
        count = count +1
        for u in range(0, len(sentences)):
            if sentences[u].__contains__(frequencyWordList[i][1]):
                inDocument = inDocument+1
        frequencyWordList[count][0] = math.log(amountSentence/(inDocument+1))
        inDocument = 0
    return frequencyWordList
    
      
def bag_of_words():
    """
    List of dictonarys that contain labels( h = not spam, s = spam )
    and the processed sentences
    """
    listeSentenceTraining = processSentence(trainingtextSentence) 
    listeSentenceTest = processSentence(testtextSentence) 
    listeWordTraining = processSpecialWords(TrainingTextWord)
    matrixTraining = creatingvector(listeSentenceTraining, listeWordTraining)
    SumofWords = countingWords(matrixTraining)

    inputTF_IDF = tf_idf(TrainingTextWord)
    matrixTrainingIDF = creatingvector(listeSentenceTraining, listeWordTraining, inputTF_IDF)
    matrixTestIDF = creatingvector(listeSentenceTest, listeWordTraining, inputTF_IDF)
    SumofWordsIDF = countingWords(matrixTestIDF)
    matrixProbIDF = probabilityCalculator(matrixTrainingIDF, listeWordTraining, SumofWordsIDF)
    SumofWordsIDF = countingWords(matrixTrainingIDF)
    LabelCountIDF = countingLabel(matrixTraining)
    predictingClass(matrixTestIDF, matrixProbIDF[0], matrixProbIDF[1], LabelCountIDF, SumofWords, listeWordTraining)

def tf_idf(TrainingTextWord):
    tf_idf_values = []
    TrainingTextWord = processSpecialWordsComplete(TrainingTextWord)
    frequencyList = termfrequency(TrainingTextWord)
    inverseList = InverseDocumentFrequency(frequencyList, trainingtextSentence)
    for i in range(0, len(frequencyList)):
        tf_idf_values.append([frequencyList[i][0] * inverseList[i][0], frequencyList[i][1]])
    
    print("TF-IDF DONE")
    return tf_idf_values
    
bag_of_words()


Process Sentence DONE
Process Sentence DONE
Process Special Words DONE
Creatin Vector DONE
Process Special Words Complete DONE
TF-IDF DONE
Creatin Vector DONE
Creatin Vector DONE
Probability Starts
Probability DONE
Email1: [{'Class': 'h', 'Text': ['sure,', 'if', 'i', 'get', 'an', 'acknowledgement', 'from', 'you', 'that', "it's", 'astoundingly', 'tactless', 'and', 'generally', 'faggy', 'to', 'demand', 'a', 'blood', 'oath', 'fo']}, 0, 3209.7422956831883, 0, 0, 3318.0149249307906, 0, 0, 0, 0, 0, 3317.0629822998676, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3319.773479369184, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3307.540743462582, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3318.1798082135624, 0, 0, 0, 3288.4713006730976, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3316.000001560549, 0, 0, 0, 3289.0767378417054, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

### Additional Experiments *(5 additional points - <span style="color: red;">Optional</span>)*